In [123]:
import boto3
import json
import pandas as pd
import time

import matplotlib.pyplot as plt
plt.style.use('seaborn')

C:\Users\voyno\AppData\Local\Temp\ipykernel_14520\3307488785.py:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [73]:
preprocessing = [
    {
        "update_col": "price", 
        "from_col": "price",
        "fn": lambda x: int(x.split(" ")[0].replace("$", "").replace(",", "")),
    },
    {
        "update_col": "score", 
        "from_col": "score",
        "fn": lambda x: float(x),
    },
    {
        "update_col": "num_tickets", 
        "from_col": "num_tickets",
        "fn": lambda x: x.split(" ")[2],
    },
    {
        "update_col": "section", 
        "from_col": "section",
        "fn": lambda x: x.replace("Section ", "").replace("Row ", "") + ", NA",
    },
    {
        "update_col": "row", 
        "from_col": "section",
        "fn": lambda x: x.split(", ")[1],
    },
    {
        "update_col": "section", 
        "from_col": "section",
        "fn": lambda x: x.split(", ")[0],
    },
]

bucket_name = "seatgeek-tickets"
table = "2023-01-16-09-26-43-941432.json"
s3 = boto3.resource('s3')

In [84]:
# get all data from s3
s3_objs = s3.Bucket(bucket_name).objects.all()
obj_keys = [obj.key for obj in s3_objs][1:]

# load data
data = {}
for key in obj_keys:
    obj = s3.Object(bucket_name, key)
    json_obj = json.loads(obj.get()['Body'].read())
    key_stem = key.replace(".json", "")
    data[key_stem] = json_obj

# concatenate data
dfs = []
for datetime, dataset in data.items():
    for game, table in dataset.items():
        df = pd.DataFrame(table)
        df['game'] = game
        df['datetime'] = datetime
        dfs.append(df)
dfs = pd.concat(dfs)

# data preprocessing
for p in preprocessing:
    dfs[p['update_col']] = dfs[p['from_col']].apply(p['fn'])

In [156]:
import numpy as np

In [173]:
section_row = dfs['section'] + dfs['row']

count = {}
for comb in section_row:
    if comb in count:
        count[comb] += 1
    else:
        count[comb] = 1

maxv = 0
maxk = ""
for k, v in count.items():
    if v > maxv:
        maxv = v
        maxk = k
print(maxv, maxk)

25 10415


In [192]:

for section in dfs['section'].unique():
    for row in dfs['row'].unique():
        for game, game_group in dfs[(dfs['section'] == section) * (dfs['row'] == '15')].groupby('game'):
            for date, date_group in game_group.groupby('datetime')['price']:
                if len(date_group) > 1:
                    print(section, row, game, date, date_group.mean())

215 J Cleveland Cavaliers 2023-01-15-16-53-33-938440 31.0
215 J Cleveland Cavaliers 2023-01-16-16-40-57-005388 33.0
215 J Houston Rockets 2023-01-15-16-53-33-938440 20.666666666666668
215 J Houston Rockets 2023-01-16-16-40-57-005388 21.666666666666668
215 J Orlando Magic(1) 2023-01-15-16-53-33-938440 18.333333333333332
215 J Orlando Magic(1) 2023-01-16-16-40-57-005388 19.0
215 9 Cleveland Cavaliers 2023-01-15-16-53-33-938440 31.0
215 9 Cleveland Cavaliers 2023-01-16-16-40-57-005388 33.0
215 9 Houston Rockets 2023-01-15-16-53-33-938440 20.666666666666668
215 9 Houston Rockets 2023-01-16-16-40-57-005388 21.666666666666668
215 9 Orlando Magic(1) 2023-01-15-16-53-33-938440 18.333333333333332
215 9 Orlando Magic(1) 2023-01-16-16-40-57-005388 19.0
215 7 Cleveland Cavaliers 2023-01-15-16-53-33-938440 31.0
215 7 Cleveland Cavaliers 2023-01-16-16-40-57-005388 33.0
215 7 Houston Rockets 2023-01-15-16-53-33-938440 20.666666666666668
215 7 Houston Rockets 2023-01-16-16-40-57-005388 21.666666666666